<a href="https://colab.research.google.com/github/ninja-marduk/ml_precipitation_prediction/blob/feature%2Fhybrid-models/models/hybrid_models_enconders_layering_w3_ST-HybridWaveStack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Entrenamiento Multi‐rama con GRU encoder–decoder y Transformer para low,
validación y forecast parametrizables, meta‐modelo XGBoost (stacking all H=1–3),
paralelización, trazabilidad y límites del departamento de Boyacá.
"""

import sys
from pathlib import Path
import warnings
import logging

# Función para verificar disponibilidad de datos para lags de precipitación
def verify_precipitation_lags(ds, required_lags=None, min_valid_ratio=0.90):
    """
    Verifica si hay suficientes datos disponibles para procesar los lags de precipitación.
    
    Args:
        ds: Dataset xarray que contiene las variables
        required_lags: Lista de lags requeridos (si None, verifica todos los disponibles)
        min_valid_ratio: Proporción mínima de datos válidos para considerar aceptable
        
    Raises:
        ValueError: Si no hay lags disponibles o si la proporción de datos válidos es insuficiente
    """
    # Lista de posibles lags de precipitación
    all_possible_lags = [
        "total_precipitation_lag1", "total_precipitation_lag2", 
        "total_precipitation_lag3", "total_precipitation_lag4",
        "total_precipitation_lag12", "total_precipitation_lag24", 
        "total_precipitation_lag36"
    ]
    
    # Determinar qué lags verificar
    lags_to_check = required_lags if required_lags else [lag for lag in all_possible_lags if lag in ds.data_vars]
    
    if not lags_to_check:
        raise ValueError("No se encontraron variables de lag de precipitación en el dataset")
    
    logger.info(f"Verificando disponibilidad de datos para {len(lags_to_check)} lags de precipitación")
    
    # Verificar cada lag
    for lag in lags_to_check:
        if lag not in ds.data_vars:
            raise ValueError(f"El lag requerido {lag} no está disponible en el dataset")
        
        # Calcular proporción de datos válidos
        lag_data = ds[lag].values
        total_elements = lag_data.size
        valid_elements = total_elements - np.isnan(lag_data).sum()
        valid_ratio = valid_elements / total_elements
        
        logger.info(f"Lag {lag}: {valid_ratio:.2%} de datos válidos ({valid_elements}/{total_elements})")
        
        # Verificar si hay suficientes datos válidos
        if valid_ratio < min_valid_ratio:
            raise ValueError(
                f"Insuficientes datos válidos para {lag}. "
                f"Disponible: {valid_ratio:.2%}, Requerido: {min_valid_ratio:.2%}"
            )
    
    logger.info("✅ Verificación de lags de precipitación completada con éxito")
    return True

# 0) Detectar entorno (Local / Colab)
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    BASE_PATH = Path("/content/drive/MyDrive/ml_precipitation_prediction")
    !pip install -q xarray netCDF4 optuna seaborn cartopy xgboost ace_tools_open cartopy
else:
    BASE_PATH = Path.cwd()
    for p in [BASE_PATH, *BASE_PATH.parents]:
        if (p/".git").exists():
            BASE_PATH = p
            break
print(f"▶️ Base path: {BASE_PATH}")

# 1) Suprimir warnings irrelevantes
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from cartopy.io import DownloadWarning
warnings.filterwarnings("ignore", category=DownloadWarning)
import tensorflow as tf
tf.get_logger().setLevel("ERROR")

# 2) Parámetros configurables
INPUT_WINDOW    = 48          # meses de entrada según análisis ACF/PACF
OUTPUT_HORIZON  = 3           # meses de validación y forecast
REF_DATE        = "2025-03"   # fecha de referencia yyyy-mm
MAX_EPOCHS      = 300
PATIENCE_ES     = 30
LR_FACTOR       = 0.5
LR_PATIENCE     = 10
DROPOUT         = 0.1

# 3) Rutas y logger
MODEL_DIR    = BASE_PATH/"models"/"output"/"trained_models"
MODEL_DIR.mkdir(parents=True, exist_ok=True)
FEATURES_NC  = BASE_PATH/"models"/"output"/"features_fusion_branches.nc"
FULL_NC      = BASE_PATH/"data"/"output"/"complete_dataset_with_features_with_clusters_elevation_with_windows.nc"
SHP_USER     = Path("/mnt/data/MGN_Departamento.shp")
BOYACA_SHP   = SHP_USER if SHP_USER.exists() else BASE_PATH/"data"/"input"/"shapes"/"MGN_Departamento.shp"
RESULTS_CSV  = MODEL_DIR/f"metrics_w{OUTPUT_HORIZON}_ref{REF_DATE}.csv"
IMAGE_DIR    = MODEL_DIR/"images"
IMAGE_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger(__name__)

# 4) Imports principales
import numpy            as np
import pandas           as pd
import xarray           as xr
import geopandas        as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import psutil
from joblib import cpu_count
from scipy.stats import skew

def print_progress(message, level=0, is_start=False, is_end=False):
    """
    Print a formatted progress message.
    
    Args:
        message: The message to print
        level: Indentation level (0, 1, 2)
        is_start: Whether this is the start of a section
        is_end: Whether this is the end of a section
    """
    prefix = ""
    if level == 0:
        if is_start:
            prefix = "🔵 "
        elif is_end:
            prefix = "✅ "
        else:
            prefix = "➡️ "
    elif level == 1:
        prefix = "   ⚪ "
    else:
        prefix = "     • "
        
    print(f"{prefix}{message}")
from tensorflow.keras.layers import (
    Input, GRU, RepeatVector, TimeDistributed, Dense,
    MultiHeadAttention, Add, LayerNormalization, Flatten
)
from tensorflow.keras.models import Model
from tensorflow.keras import callbacks

# 5) Recursos hardware
CORES     = cpu_count()
AVAIL_RAM = psutil.virtual_memory().available / (1024**3)
gpus      = tf.config.list_physical_devices("GPU")
USE_GPU   = bool(gpus)
if USE_GPU:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    logger.info(f"🖥 GPU disponible: {gpus[0].name}")
else:
    tf.config.threading.set_inter_op_parallelism_threads(CORES)
    tf.config.threading.set_intra_op_parallelism_threads(CORES)
    logger.info(f"⚙ CPU cores: {CORES}, RAM libre: {AVAIL_RAM:.1f} GB")

# 6) Modelos y utilitarios
def evaluate_metrics(y_true, y_pred):
    # Filtrar NaNs para robustez
    mask = ~(np.isnan(y_true) | np.isnan(y_pred))
    y_true, y_pred = y_true[mask], y_pred[mask]
    
    # Verificar que hay suficientes datos válidos
    if len(y_true) < 10:
        logger.warning(f"Insuficientes datos válidos para calcular métricas: {len(y_true)} < 10")
        return np.nan, np.nan, np.nan, np.nan
        
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    
    # Evitar división por cero en MAPE
    nonzero_mask = y_true != 0
    if np.sum(nonzero_mask) > 10:
        mape = np.mean(np.abs((y_true[nonzero_mask] - y_pred[nonzero_mask])/(y_true[nonzero_mask] + 1e-5))) * 100
    else:
        mape = np.nan
    
    # Cálculo de R2 solo si hay suficiente varianza
    var = np.var(y_true)
    if var > 1e-10:
        r2 = 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)
    else:
        r2 = np.nan
        
    return rmse, mae, mape, r2

def check_nans(arr, name="array"):
    """Verifica si hay NaNs en un array y retorna un resumen"""
    nan_count = np.isnan(arr).sum()
    total_count = arr.size
    nan_percentage = (nan_count / total_count) * 100 if total_count > 0 else 0
    
    return {
        "name": name,
        "nan_count": nan_count,
        "total_elements": total_count,
        "nan_percentage": nan_percentage,
        "has_nans": nan_count > 0
    }

def replace_nans(arr, strategy="mean", fill_value=None):
    """Reemplaza valores NaN en un array usando diferentes estrategias"""
    if not np.isnan(arr).any():
        return arr
    
    # Crear copia para no modificar el original
    arr_copy = arr.copy()
    
    if strategy == "mean":
        fill = np.nanmean(arr)
    elif strategy == "median":
        fill = np.nanmedian(arr)
    elif strategy == "zero":
        fill = 0.0
    elif strategy == "constant":
        fill = 0.0 if fill_value is None else fill_value
    elif strategy == "interpolate":
        # Para series temporales - simple interpolación lineal
        if arr_copy.ndim == 1:
            mask = np.isnan(arr_copy)
            if np.all(mask):  # Si todo es NaN
                return np.zeros_like(arr_copy)
            if not np.any(~mask):  # Si no hay valores válidos
                return np.zeros_like(arr_copy)
            arr_copy[mask] = np.interp(
                np.flatnonzero(mask), 
                np.flatnonzero(~mask), 
                arr_copy[~mask]
            )
            return arr_copy
        else:
            # Aplanar, interpolar y restaurar forma
            original_shape = arr_copy.shape
            arr_flat = arr_copy.reshape(-1)
            mask = np.isnan(arr_flat)
            if np.all(mask) or not np.any(~mask):  # Si todo es NaN o no hay valores válidos
                return np.zeros_like(arr_copy)
            arr_flat[mask] = np.interp(
                np.flatnonzero(mask), 
                np.flatnonzero(~mask), 
                arr_flat[~mask]
            )
            return arr_flat.reshape(original_shape)
    else:
        raise ValueError(f"Estrategia '{strategy}' no reconocida")
    
    # Reemplazar NaNs
    arr_copy[np.isnan(arr_copy)] = fill
    return arr_copy

class ScalerNaN:
    """StandardScaler que maneja NaNs de forma segura"""
    def __init__(self):
        self.mean_ = None
        self.scale_ = None
        
    def fit(self, X):
        self.mean_ = np.nanmean(X, axis=0)
        # Usar nanvar con ddof=0 para consistencia con StandardScaler
        self.var_ = np.nanvar(X, axis=0, ddof=0)
        # Evitar división por cero
        self.var_[self.var_ < 1e-10] = 1.0
        self.scale_ = np.sqrt(self.var_)
        return self
        
    def transform(self, X):
        X_transformed = X.copy()
        # Mantener la estructura dimensional para el broadcasting correcto
        # Iterar sobre cada fila para mantener la compatibilidad dimensional
        for i in range(X.shape[0]):
            row_mask = ~np.isnan(X[i, :])
            if np.any(row_mask):
                X_transformed[i, row_mask] = (X[i, row_mask] - self.mean_[row_mask]) / self.scale_[row_mask]
        return X_transformed
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)
    
    def inverse_transform(self, X):
        X_inv = X.copy()
        # Usar la misma lógica de iteración para inversa
        for i in range(X.shape[0]):
            row_mask = ~np.isnan(X[i, :])
            if np.any(row_mask):
                X_inv[i, row_mask] = X[i, row_mask] * self.scale_[row_mask] + self.mean_[row_mask]
        return X_inv

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, X, Y, batch_size=32, **kwargs):
        super().__init__(**kwargs)
        self.X, self.Y = X.astype(np.float32), Y.astype(np.float32)
        self.batch_size = batch_size
    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))
    def __getitem__(self, idx):
        sl = slice(idx*self.batch_size, (idx+1)*self.batch_size)
        return self.X[sl], self.Y[sl]

class TrainingProgressCallback(tf.keras.callbacks.Callback):
    def __init__(self, model_name, total_epochs):
        super().__init__()
        self.model_name = model_name
        self.total_epochs = total_epochs
        self.current_epoch = 0
        
    def on_epoch_begin(self, epoch, logs=None):
        self.current_epoch = epoch
        
    def on_epoch_end(self, epoch, logs=None):
        # Update progress after each epoch
        loss = logs.get('loss', 0.0)
        val_loss = logs.get('val_loss', 0.0)
        progress = (epoch + 1) / self.total_epochs * 100
        
        # Print progress information
        print_progress(
            f"Entrenamiento {self.model_name}: Época {epoch+1}/{self.total_epochs} " +
            f"({progress:.1f}%) - loss: {loss:.4f} - val_loss: {val_loss:.4f}",
            level=2
        )

# GRU-ED y Transformer-ED builders
from tensorflow.keras import backend as K

def build_gru_ed(input_shape, horizon, n_cells, latent=128, dropout=DROPOUT):
    inp = Input(shape=input_shape)
    x = GRU(latent, dropout=dropout)(inp)
    x = RepeatVector(horizon)(x)
    x = GRU(latent, dropout=dropout, return_sequences=True)(x)
    out = TimeDistributed(Dense(n_cells))(x)
    m = Model(inp, out)
    m.compile(optimizer="adam", loss="mse")
    return m


def build_transformer_ed(input_shape, horizon, n_cells,
                         head_size=64, num_heads=4, ff_dim=256, dropout=0.1):
    inp = Input(shape=input_shape)
    attn = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(inp, inp)
    x = Add()([inp, attn])
    x = LayerNormalization(epsilon=1e-6)(x)
    ff = Dense(ff_dim, activation="relu")(x)
    ff = Dense(input_shape[-1])(ff)
    x = Add()([x, ff])
    x = LayerNormalization(epsilon=1e-6)(x)
    x = Flatten()(x)
    x = Dense(horizon * n_cells)(x)
    out = K.reshape(x, (-1, horizon, n_cells))
    m = Model(inp, out)
    m.compile(optimizer="adam", loss="mse")
    return m


def build_gru_ed_low(input_shape, horizon, n_cells,
                     latent=256, dropout=0.1, use_transformer=True):
    if use_transformer:
        try:
            return build_transformer_ed(input_shape, horizon, n_cells,
                                        head_size=64, num_heads=4,
                                        ff_dim=512, dropout=dropout)
        except tf.errors.ResourceExhaustedError:
            logger.warning("OOM Transformer → usando GRU‐ED para low-branch")
    return build_gru_ed(input_shape, horizon, n_cells,
                        latent=latent, dropout=dropout)


def build_gru_ed_medium_high(input_shape, horizon, n_cells, latent=128, dropout=0.1, use_transformer=True):
    try:
        if use_transformer:
            return build_transformer_ed(input_shape, horizon, n_cells,
                                        head_size=64, num_heads=4,
                                        ff_dim=512, dropout=dropout)
        else:
            return build_gru_ed(input_shape, horizon, n_cells,
                                latent=latent, dropout=dropout)
    except tf.errors.ResourceExhaustedError:
        logger.warning("OOM Transformer → usando GRU‐ED para medium/high")
        return build_gru_ed(input_shape, horizon, n_cells,
                            latent=latent, dropout=dropout)

# 7) Carga datos y shapefile
logger.info("📂 Cargando datasets…")
ds_full = xr.open_dataset(FULL_NC)
ds_feat = xr.open_dataset(FEATURES_NC)
boyaca_gdf = gpd.read_file(BOYACA_SHP)
if boyaca_gdf.crs is None:
    boyaca_gdf.set_crs(epsg=4326, inplace=True)
else:
    boyaca_gdf = boyaca_gdf.to_crs(epsg=4326)

times      = ds_full.time.values.astype("datetime64[M]")
user_ref   = np.datetime64(REF_DATE, "M")
last_avail = times[-1]
ref = last_avail if user_ref>last_avail else user_ref

val_dates = [
    str(ref),
    str((ref - np.timedelta64(1,'M')).astype("datetime64[M]")),
    str((ref - np.timedelta64(2,'M')).astype("datetime64[M]"))
]
fc_dates  = [str((ref + np.timedelta64(i+1,'M')).astype("datetime64[M]")) for i in range(OUTPUT_HORIZON)]

idx_ref = int(np.where(times == ref)[0][0])
lat     = ds_full.latitude.values
lon     = ds_full.longitude.values
METHODS = ["CEEMDAN","TVFEMD","FUSION"]
BRANCHES= ["high","medium","low"]

# Verificar qué lags de precipitación están disponibles 
LAG_FEATURES = [
    "total_precipitation_lag1",
    "total_precipitation_lag2", 
    "total_precipitation_lag3",
    "total_precipitation_lag4",
    "total_precipitation_lag12",
    "total_precipitation_lag24",
    "total_precipitation_lag36"
]
available_lags = [lag for lag in LAG_FEATURES if lag in ds_full.data_vars]
logger.info(f"📊 Lags de precipitación disponibles: {available_lags}")

all_metrics = []
preds_store = {}
true_store  = {}
histories   = {}

# callbacks
es_cb = callbacks.EarlyStopping("val_loss", patience=PATIENCE_ES, restore_best_weights=True)
lr_cb = callbacks.ReduceLROnPlateau("val_loss", factor=LR_FACTOR, patience=LR_PATIENCE, min_lr=1e-6)

# 8) Bucle principal
print_progress(f"Iniciando procesamiento de {len(METHODS)} métodos × {len(BRANCHES)} branches con manejo robusto de NaNs", is_start=True)
total_combinations = len(METHODS) * len(BRANCHES)
processed = 0

for method in METHODS:
    for branch in BRANCHES:
        processed += 1
        name = f"{method}_{branch}"
        if name not in ds_feat.data_vars:
            print_progress(f"({processed}/{total_combinations}) ⚠️ {name} no existe, salteando...")
            continue
            
        print_progress(f"({processed}/{total_combinations}) Procesando {name}", is_start=True)
        try:
            # extraer y aplanar
            Xarr = ds_feat[name].values            # (T, ny, nx)
            Yarr = ds_full["total_precipitation"].values  # (T, ny, nx)
            
            # Verificar NaNs iniciales
            x_summary = check_nans(Xarr, f"Entrada {name}")
            y_summary = check_nans(Yarr, f"Objetivo {name}")
            
            if x_summary["has_nans"]:
                print_progress(f"⚠️ Detectados {x_summary['nan_count']} NaNs en entrada {name} ({x_summary['nan_percentage']:.2f}%)", level=1)
                Xarr = replace_nans(Xarr, strategy="interpolate")
                print_progress(f"NaNs reemplazados usando interpolación", level=2)
            
            if y_summary["has_nans"]:
                print_progress(f"⚠️ Detectados {y_summary['nan_count']} NaNs en objetivo {name} ({y_summary['nan_percentage']:.2f}%)", level=1)
                Yarr = replace_nans(Yarr, strategy="interpolate")
                print_progress(f"NaNs reemplazados usando interpolación", level=2)
            
            T, ny, nx = Xarr.shape
            n_cells   = ny * nx

            Xfull = Xarr.reshape(T, n_cells)
            yfull = Yarr.reshape(T, n_cells)

            # ventanas
            Nw = T - INPUT_WINDOW - OUTPUT_HORIZON + 1
            if Nw <= 0:
                print_progress(f"❌ Ventanas insuficientes para {name}, continuando con el siguiente", level=1)
                continue

            print_progress(f"Generando {Nw} ventanas para {name}", level=1)
            
            Xs = np.stack([Xfull[i : i+INPUT_WINDOW] for i in range(Nw)], axis=0)
            ys = np.stack([yfull[i+INPUT_WINDOW : i+INPUT_WINDOW+OUTPUT_HORIZON]
                           for i in range(Nw)], axis=0)

            # ========================================================================
            # IMPLEMENTACIÓN ROBUSTA DE LAGS DE PRECIPITACIÓN
            # ========================================================================
            print_progress(f"Procesando lags de precipitación para {name} de forma robusta", level=1)

            # Preparar lista para características adicionales
            features_to_add = []

            # sin/cos para low
            if branch == "low":
                months = pd.to_datetime(ds_full.time.values).month.values
                s = np.sin(2 * np.pi * months/12)
                c = np.cos(2 * np.pi * months/12)
                Ss = np.stack([s[i : i+INPUT_WINDOW] for i in range(Nw)], axis=0)
                Cs = np.stack([c[i : i+INPUT_WINDOW] for i in range(Nw)], axis=0)
                Ss = np.repeat(Ss[:,:,None], n_cells, axis=2)
                Cs = np.repeat(Cs[:,:,None], n_cells, axis=2)
                features_to_add.extend([Ss, Cs])
                logger.info(f"✓ Agregadas características estacionales sin/cos para branch {branch}")

            # Agregar lags de precipitación como features adicionales (manejo robusto)
            if available_lags:
                logger.info(f"🔄 Agregando {len(available_lags)} lags de precipitación al branch {branch}")
                for lag_var in available_lags:
                    # Obtener datos y verificar NaNs
                    lag_data = ds_full[lag_var].values
                    lag_summary = check_nans(lag_data, f"Lag {lag_var}")
                    
                    # Manejar NaNs según el porcentaje
                    if lag_summary["has_nans"]:
                        print_progress(f"⚠️ {lag_var}: {lag_summary['nan_count']} NaNs ({lag_summary['nan_percentage']:.2f}%)", level=2)
                        if lag_summary["nan_percentage"] < 5:
                            lag_data = replace_nans(lag_data, strategy="interpolate")
                            print_progress(f"NaNs interpolados en {lag_var}", level=2)
                        elif lag_summary["nan_percentage"] < 20:
                            lag_data = replace_nans(lag_data, strategy="mean")
                            print_progress(f"NaNs reemplazados con media en {lag_var}", level=2)
                        else:
                            lag_data = replace_nans(lag_data, strategy="zero")
                            print_progress(f"⚠️ Demasiados NaNs en {lag_var}, reemplazando con ceros", level=2)
                    
                    lag_full = lag_data.reshape(T, n_cells)
                    lag_windows = np.stack([lag_full[i : i+INPUT_WINDOW] for i in range(Nw)], axis=0)
                    features_to_add.append(lag_windows)
                logger.info(f"✓ Lags procesados robustamente: {available_lags}")

            # Concatenar todas las características
            if features_to_add:
                Xs = np.concatenate([Xs] + features_to_add, axis=2)
                n_feats = Xs.shape[2]
                print_progress(f"Estructura de features: {Xs.shape} ({n_feats} features totales)", level=1)
            else:
                n_feats = n_cells
                print_progress(f"Sin features adicionales: {Xs.shape}", level=1)

            # Verificar NaNs después del procesamiento
            xs_processed_summary = check_nans(Xs, "Features procesados")
            if xs_processed_summary["has_nans"]:
                print_progress(f"⚠️ Aún hay {xs_processed_summary['nan_count']} NaNs después del procesamiento, reemplazando", level=1)
                Xs = replace_nans(Xs, strategy="mean")
                
            # escalado robusto
            print_progress("Aplicando escalado robusto de datos", level=1)
            # Usar ScalerNaN para manejar valores NaN correctamente
            scX = ScalerNaN().fit(Xs.reshape(-1, n_feats))
            scY = ScalerNaN().fit(ys.reshape(-1, n_cells))
            
            Xs_s = scX.transform(Xs.reshape(-1, n_feats)).reshape(Xs.shape)
            ys_s = scY.transform(ys.reshape(-1, n_cells)).reshape(ys.shape)
            
            # Verificar NaNs después del escalado
            xs_scaled_summary = check_nans(Xs_s, "Features escalados")
            ys_scaled_summary = check_nans(ys_s, "Objetivos escalados")
            
            if xs_scaled_summary["has_nans"] or ys_scaled_summary["has_nans"]:
                print_progress("⚠️ Hay NaNs después del escalado, reemplazando con ceros", level=1)
                # Reemplazar NaNs restantes con ceros
                Xs_s = np.nan_to_num(Xs_s, nan=0.0)
                ys_s = np.nan_to_num(ys_s, nan=0.0)

            # partición centrada en REF_DATE
            k_ref = np.clip(idx_ref - INPUT_WINDOW + 1, 0, Nw-1)
            i0    = np.clip(k_ref - (OUTPUT_HORIZON-1), 0, Nw-OUTPUT_HORIZON)

            X_tr, y_tr = Xs_s[:i0], ys_s[:i0]
            X_va, y_va = Xs_s[i0 : i0+OUTPUT_HORIZON], ys_s[i0 : i0+OUTPUT_HORIZON]

            # cargar/entrenar
            model_path = MODEL_DIR/f"{name}_w{OUTPUT_HORIZON}_ref{ref}.keras"
            if model_path.exists():
                print_progress(f"Cargando modelo existente: {model_path.name}", level=1)
                model = tf.keras.models.load_model(str(model_path), compile=False)
                model.compile(optimizer="adam", loss="mse")
            else:
                print_progress(f"Creando nuevo modelo para {name}", level=1)
                if branch == "low":
                    model = build_gru_ed_low((INPUT_WINDOW, n_feats), OUTPUT_HORIZON, n_cells)
                    print_progress(f"Modelo low-branch creado: {model.__class__.__name__}", level=2)
                else:
                    model = build_gru_ed_medium_high((INPUT_WINDOW, n_feats), OUTPUT_HORIZON, n_cells)
                    print_progress(f"Modelo {branch}-branch creado: {model.__class__.__name__}", level=2)

                # Crear callback de progreso personalizado
                progress_cb = TrainingProgressCallback(name, MAX_EPOCHS)
                
                # Mostrar resumen de datos de entrenamiento
                print_progress(f"Entrenando con {len(X_tr)} muestras, validando con {len(X_va)} muestras", level=1)
                print_progress(f"X_train: {X_tr.shape}, y_train: {y_tr.shape}", level=2)
                print_progress(f"X_val: {X_va.shape}, y_val: {y_va.shape}", level=2)
                
                # Entrenamiento con barra de progreso
                print_progress(f"Iniciando entrenamiento para {name}", level=1)
                hist = model.fit(
                    DataGenerator(X_tr, y_tr),
                    validation_data=DataGenerator(X_va, y_va),
                    epochs=MAX_EPOCHS,
                    callbacks=[es_cb, lr_cb, progress_cb],
                    verbose=0  # Desactivamos verbose integrado ya que tenemos progress_cb
                )
                
                print_progress(f"Guardando modelo en {model_path.name}", level=1)
                model.save(str(model_path))
                histories[name] = hist.history
                
                # Mostrar información del entrenamiento
                print_progress(f"Entrenamiento completado en {len(hist.history['loss'])} épocas", level=1)
                print_progress(f"Loss inicial: {hist.history['loss'][0]:.4f}, Loss final: {hist.history['loss'][-1]:.4f}", level=2)
                print_progress(f"Val-loss inicial: {hist.history['val_loss'][0]:.4f}, Val-loss final: {hist.history['val_loss'][-1]:.4f}", level=2)

            # validación H=1..H
            print_progress(f"Generando predicciones de validación para {name}", level=1)
            preds = model.predict(X_va, verbose=0).reshape(OUTPUT_HORIZON, OUTPUT_HORIZON, n_cells)
            for h in range(OUTPUT_HORIZON):
                date_val = val_dates[h]
                pm_flat  = preds[h,0]
                tm_flat  = y_va[h,0]
                pm = scY.inverse_transform(pm_flat.reshape(1,-1))[0].reshape(ny,nx)
                tm = scY.inverse_transform(tm_flat.reshape(1,-1))[0].reshape(ny,nx)
                rmse, mae, mape, r2 = evaluate_metrics(tm.ravel(), pm.ravel())
                all_metrics.append({
                    "model": name, "branch": branch, "horizon": h+1,
                    "type":"validation", "date": date_val,
                    "RMSE": rmse, "MAE": mae, "MAPE": mape, "R2": r2
                })
                preds_store[(name,date_val)] = pm
                true_store[(name,date_val)]  = tm

            # forecast
            print_progress(f"Generando predicciones de forecast para {name}", level=1)
            X_fc = Xs_s[k_ref : k_ref+1]
            fc_s = model.predict(X_fc, verbose=0)[0]
            FC   = scY.inverse_transform(fc_s)
            for h in range(OUTPUT_HORIZON):
                date_fc = fc_dates[h]
                all_metrics.append({
                    "model": name, "branch": branch, "horizon": h+1,
                    "type":"forecast", "date": date_fc,
                    "RMSE": np.nan, "MAE": np.nan, "MAPE": np.nan, "R2": np.nan
                })
                preds_store[(name,date_fc)] = FC[h].reshape(ny,nx)

        except Exception as e:
            print_progress(f"‼️ Error en {name}: {str(e)}", level=1)
            logger.exception(f"Error en {name}, continuo…")
            continue

print_progress("Procesamiento de todos los modelos completado", is_end=True)

# 9) Guardar métricas y mostrar tabla
print_progress("Guardando métricas y generando tablas", is_start=True)
dfm = pd.DataFrame(all_metrics)
dfm.to_csv(RESULTS_CSV, index=False)
import ace_tools_open as tools
import cartopy.crs as ccrs
tools.display_dataframe_to_user(name=f"Metrics_w{OUTPUT_HORIZON}_ref{ref}", dataframe=dfm)
print_progress(f"Métricas guardadas en {RESULTS_CSV}", is_end=True)

# 10) Curvas de entrenamiento
print_progress("Generando curvas de entrenamiento", is_start=True)
for name, hist in histories.items():
    plt.figure(figsize=(6,4))
    plt.plot(hist["loss"],  label="train")
    plt.plot(hist["val_loss"],label="val")
    plt.title(f"Loss curve: {name}")
    plt.xlabel("Epoch"); plt.ylabel("MSE")
    plt.legend(); plt.tight_layout(); plt.show()
print_progress("Visualizaciones de curvas de entrenamiento completadas", is_end=True)

# 10bis) True vs Predicted por rama y horizonte
for branch in BRANCHES:
    for h in range(1, OUTPUT_HORIZON+1):
        plt.figure(figsize=(5,5))
        for method in METHODS:
            key = f"{method}_{branch}"
            date_val = val_dates[h-1]
            if (key, date_val) in preds_store and (key, date_val) in true_store:
                y_true = true_store[(key, date_val)].ravel()
                y_pred = preds_store[(key, date_val)].ravel()
                plt.scatter(y_true, y_pred, alpha=0.3, s=2, label=method)
        lims = [0, max(plt.xlim()[1], plt.ylim()[1])]
        plt.plot(lims, lims, 'k--')
        plt.xlabel("True"); plt.ylabel("Predicted")
        plt.title(f"True vs Pred — {branch}, H={h}")
        plt.legend(); plt.tight_layout(); plt.show()

# 11) Mapas 3×3 validación H=1
xmin, ymin, xmax, ymax = boyaca_gdf.total_bounds
for date_val in val_dates:
    arrs = [preds_store[(f"{m}_{b}",date_val)].ravel()
            for m in METHODS for b in BRANCHES
            if (f"{m}_{b}",date_val) in preds_store]
    if not arrs:
        logger.warning(f"No hay predicciones para {date_val}, salto plot.")
        continue
    vmin, vmax = np.min(arrs), np.max(arrs)
    fig, axs = plt.subplots(3,3, figsize=(12,12), subplot_kw={"projection":ccrs.PlateCarree()})
    fig.suptitle(f"Validación H=1 — {date_val}", fontsize=16)
    for i, b in enumerate(BRANCHES):
        for j, m in enumerate(METHODS):
            ax = axs[i,j]
            ax.set_extent([xmin, xmax, ymin, ymax], ccrs.PlateCarree())
            ax.add_geometries(boyaca_gdf.geometry, ccrs.PlateCarree(),
                              edgecolor="black", facecolor="none", linewidth=1)
            key = (f"{m}_{b}", date_val)
            if key in preds_store:
                pcm = ax.pcolormesh(lon, lat, preds_store[key],
                                    vmin=vmin, vmax=vmax,
                                    transform=ccrs.PlateCarree(), cmap="Blues")
            ax.set_title(f"{m}_{b}")
    fig.colorbar(pcm, ax=axs, orientation="horizontal",
                 fraction=0.05, pad=0.04, label="Precipitación (mm)")
    fig.savefig(IMAGE_DIR/f"val_H1_{date_val}.png", dpi=150); plt.show()

    arrs_mape = [
        np.clip(np.abs((true_store[k] - preds_store[k])/(true_store[k]+1e-5))*100,0,200).ravel()
        for k in preds_store if k[1]==date_val and k in true_store
    ]
    if not arrs_mape: continue
    vmin2, vmax2 = 0, np.max(arrs_mape)
    fig, axs = plt.subplots(3,3, figsize=(12,12), subplot_kw={"projection":ccrs.PlateCarree()})
    fig.suptitle(f"MAPE H=1 — {date_val}", fontsize=16)
    for i, b in enumerate(BRANCHES):
        for j, m in enumerate(METHODS):
            ax = axs[i,j]
            ax.set_extent([xmin, xmax, ymin, ymax], ccrs.PlateCarree())
            ax.add_geometries(boyaca_gdf.geometry, ccrs.PlateCarree(),
                              edgecolor="black", facecolor="none", linewidth=1)
            key = (f"{m}_{b}", date_val)
            if key in preds_store and key in true_store:
                mmap = np.clip(np.abs((true_store[key] - preds_store[key])/(true_store[key]+1e-5))*100,0,200)
                pcm2 = ax.pcolormesh(lon, lat, mmap,
                                     vmin=vmin2, vmax=vmax2,
                                     transform=ccrs.PlateCarree(), cmap="Reds")
            ax.set_title(f"{m}_{b}")
    fig.colorbar(pcm2, ax=axs, orientation="horizontal",
                 fraction=0.05, pad=0.04, label="MAPE (%)")
    fig.savefig(IMAGE_DIR/f"mape_H1_{date_val}.png", dpi=150); plt.show()

# 13) META‐MODELOS XGB stacking H=1-3 (retraining con 9 features)
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 13.0) Preparar X_meta completo para cada horizonte y retrain modelos
print_progress("Iniciando meta-modelos XGB de stacking H=1-3", is_start=True)
for h in range(1, OUTPUT_HORIZON+1):
    date = val_dates[h-1]
    print_progress(f"Entrenando meta-modelo XGB para horizonte {h}, fecha {date}", level=1)
    
    # Extraer features (3 preds + elev stats + slope + aspect)
    print_progress(f"Preparando datos para H={h}", level=2)
    preds = [preds_store[(f"FUSION_{b}", date)].ravel() for b in ['low','medium','high']]
    elev_flat   = ds_full['elevation'].values.ravel()
    slope_flat  = ds_full['slope'].values.ravel()
    aspect_flat = ds_full['aspect'].values.ravel()
    # Estadísticos de elevación
    mean_e = elev_flat.mean(); std_e = elev_flat.std(); skew_e = skew(elev_flat)
    elev_stats = np.vstack([
        np.full_like(elev_flat, mean_e),
        np.full_like(elev_flat, std_e),
        np.full_like(elev_flat, skew_e)
    ]).T
    # Construir X_meta y y_true
    X_meta = np.column_stack(preds + [elev_stats, slope_flat, aspect_flat])
    y_true = true_store[("FUSION_low", date)].ravel()
    
    # Mostrar dimensiones
    print_progress(f"X_meta shape: {X_meta.shape}, y_true shape: {y_true.shape}", level=2)
    
    # Train/test split
    X_tr, X_te, y_tr, y_te = train_test_split(X_meta, y_true, test_size=0.2, random_state=42)
    print_progress(f"Split: train={X_tr.shape[0]} muestras, test={X_te.shape[0]} muestras", level=2)
    
    # Ajustar modelo con todas las features
    print_progress(f"Entrenando XGBoost para H={h}", level=2, is_start=True)
    xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
    xgb.fit(X_tr, y_tr)
    
    # Evaluar en conjunto de prueba
    y_pred = xgb.predict(X_te)
    test_rmse = np.sqrt(mean_squared_error(y_te, y_pred))
    print_progress(f"XGB H={h} entrenado. Test RMSE: {test_rmse:.4f}", level=2, is_end=True)
    
    # Guardar modelo retrained
    model_path = str(MODEL_DIR/f"xgb_all_H{h}_{ref}_9features.json")
    print_progress(f"Guardando modelo en {model_path}", level=2)
    xgb.save_model(model_path)

print_progress("Meta-modelos XGB entrenados y guardados", is_end=True)

# 13.2) Scatter, mapas y métrica final
print_progress("Generando visualizaciones y métricas finales", is_start=True)
fig_sc, axs_sc = plt.subplots(1, OUTPUT_HORIZON, figsize=(6*OUTPUT_HORIZON,5))
for idx_h, h in enumerate(range(1, OUTPUT_HORIZON+1)):
    date = val_dates[h-1]
    mdl_path = MODEL_DIR/f"xgb_all_H{h}_{ref}_9features.json"
    if mdl_path.exists():
        xgb = XGBRegressor(); xgb.load_model(str(mdl_path))
        print_progress(f"Cargado modelo XGB para H={h}", level=1)
        
        # Recolectar X_meta completo
        preds = [preds_store[(f"FUSION_{b}", date)].ravel() for b in ['low','medium','high']]
        elev_flat   = ds_full['elevation'].values.ravel()
        slope_flat  = ds_full['slope'].values.ravel()
        aspect_flat = ds_full['aspect'].values.ravel()
        mean_e = elev_flat.mean(); std_e = elev_flat.std(); skew_e = skew(elev_flat)
        
def xgb_predict_full(model, X):
    """
    Make predictions with an XGBoost model, handling memory constraints and NaNs.
    
    Args:
        model: The XGBoost model
        X: Input features
        
    Returns:
        Predictions for all samples
    """
    # Handle NaNs in input
    has_nans = np.isnan(X).any()
    if has_nans:
        print_progress(f"⚠️ Detectados NaNs en entrada de XGB, reemplazando con valores medios", level=2)
        # Replace NaNs with column means
        X = np.copy(X)  # Create a copy to avoid modifying the original
        for col in range(X.shape[1]):
            col_data = X[:, col]
            if np.isnan(col_data).any():
                col_mean = np.nanmean(col_data)
                X[np.isnan(X[:, col]), col] = col_mean
    
    # Check if we need to batch the predictions due to memory constraints
    batch_size = 100000  # Adjust based on available memory
    if X.shape[0] > batch_size:
        # Batch predictions to avoid memory issues
        n_batches = int(np.ceil(X.shape[0] / batch_size))
        preds = []
        for i in range(n_batches):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, X.shape[0])
            try:
                batch_preds = model.predict(X[start_idx:end_idx])
                preds.append(batch_preds)
            except Exception as e:
                print_progress(f"Error en predicción batch {i}: {str(e)}", level=1)
                # Intentar con DMatrix como fallback
                try:
                    import xgboost as xgb
                    dmatrix = xgb.DMatrix(X[start_idx:end_idx])
                    batch_preds = model.predict(dmatrix)
                    preds.append(batch_preds)
                except Exception as e2:
                    print_progress(f"Error crítico en predicción: {str(e2)}", level=1)
                    # Retornar arrays de cero en caso de error irrecuperable
                    preds.append(np.zeros(end_idx - start_idx))
        return np.concatenate(preds)
    else:
        # Make predictions in one go
        try:
            return model.predict(X)
        except Exception as e:
            print_progress(f"Error en predicción: {str(e)}", level=1)
            # Intentar con DMatrix como fallback
            try:
                import xgboost as xgb
                dmatrix = xgb.DMatrix(X)
                return model.predict(dmatrix)
            except Exception as e2:
                print_progress(f"Error crítico en predicción: {str(e2)}", level=1)
                return np.zeros(X.shape[0])

# Meta metrics list
meta_metrics_all = []

# 13.2.1) Generate scatter plots and calculate metrics
for idx_h, h in enumerate(range(1, OUTPUT_HORIZON+1)):
    date = val_dates[h-1]
    mdl_path = MODEL_DIR/f"xgb_all_H{h}_{ref}_9features.json"
    if mdl_path.exists():
        xgb = XGBRegressor(); xgb.load_model(str(mdl_path))
        print_progress(f"Cargado modelo XGB para H={h}", level=1)
        
        # Recolectar X_meta completo con manejo robusto de NaNs
        preds = [preds_store[(f"FUSION_{b}", date)].ravel() for b in ['low','medium','high']]
        
        # Verificar NaNs en predicciones base
        for i, b in enumerate(['low','medium','high']):
            pred_summary = check_nans(preds[i], f"Predicción FUSION_{b}")
            if pred_summary["has_nans"]:
                print_progress(f"Reemplazando {pred_summary['nan_count']} NaNs en predicciones de FUSION_{b}", level=2)
                preds[i] = replace_nans(preds[i], strategy="mean")
        
        elev_flat = ds_full['elevation'].values.ravel()
        slope_flat = ds_full['slope'].values.ravel()
        aspect_flat = ds_full['aspect'].values.ravel()
        mean_e = elev_flat.mean(); std_e = elev_flat.std(); skew_e = skew(elev_flat)
        
        # Verificar NaNs en características topográficas
        for arr, name in zip([elev_flat, slope_flat, aspect_flat], ['elevation', 'slope', 'aspect']):
            topo_summary = check_nans(arr, name)
            if topo_summary["has_nans"]:
                print_progress(f"Reemplazando {topo_summary['nan_count']} NaNs en {name}", level=2)
                if name == 'elevation':
                    elev_flat = replace_nans(elev_flat, strategy="mean")
                elif name == 'slope':
                    slope_flat = replace_nans(slope_flat, strategy="mean")
                elif name == 'aspect':
                    aspect_flat = replace_nans(aspect_flat, strategy="mean")
        
        elev_stats = np.vstack([
            np.full_like(elev_flat, mean_e),
            np.full_like(elev_flat, std_e),
            np.full_like(elev_flat, skew_e)
        ]).T
        
        X_meta = np.column_stack(preds + [elev_stats, slope_flat, aspect_flat])
        ytrue = true_store[("FUSION_low", date)].ravel()
        
        # Verificar y manejar NaNs en objetivo
        ytrue_summary = check_nans(ytrue, "Objetivo verdadero")
        if ytrue_summary["has_nans"]:
            print_progress(f"Reemplazando {ytrue_summary['nan_count']} NaNs en objetivo verdadero", level=2)
            ytrue = replace_nans(ytrue, strategy="mean")
        
        # Predicción robusta
        ypred = xgb_predict_full(xgb, X_meta)
        
        # Scatter
        ax = axs_sc[idx_h]
        ax.scatter(ytrue, ypred, alpha=0.3, s=2)
        lims = [min(ytrue.min(), ypred.min()), max(ytrue.max(), ypred.max())]
        ax.plot(lims, lims, 'k--')
        ax.set_title(f"XGB H={h} — {date}")
        ax.set_xlabel("True"); ax.set_ylabel("Predicted")
        
        # Métricas robustas
        rm, ma, maP, r2 = evaluate_metrics(ytrue, ypred)
        meta_metrics_all.append({
            'horizon':h, 'date':date,
            'RMSE':rm, 'MAE':ma, 'MAPE':maP, 'R2':r2,
            'valid_data_pct': 100 - (np.isnan(ytrue).sum() / len(ytrue) * 100)
        })
    else:
        axs_sc[idx_h].text(0.5,0.5,f"No model H={h}",ha='center',va='center')
plt.tight_layout(); plt.show()

# con modelo retrained (con manejo robusto de NaNs)
for h in range(1, OUTPUT_HORIZON+1):
    date = val_dates[h-1]
    mdl_path = MODEL_DIR/f"xgb_all_H{h}_{ref}_9features.json"
    if not mdl_path.exists():
        continue
    
    try:
        xgb = XGBRegressor(); xgb.load_model(str(mdl_path))
        
        # Reconstruir X_meta con manejo robusto
        preds = [preds_store[(f"FUSION_{b}", date)].ravel() for b in ['low','medium','high']]
        # Manejar NaNs en predicciones
        for i, b in enumerate(['low','medium','high']):
            if np.isnan(preds[i]).any():
                preds[i] = replace_nans(preds[i], strategy="mean")
                
        elev_flat = ds_full['elevation'].values.ravel()
        slope_flat = ds_full['slope'].values.ravel()
        aspect_flat = ds_full['aspect'].values.ravel()
        
        # Manejar NaNs en características topográficas
        if np.isnan(elev_flat).any():
            elev_flat = replace_nans(elev_flat, strategy="mean")
        if np.isnan(slope_flat).any():
            slope_flat = replace_nans(slope_flat, strategy="mean")
        if np.isnan(aspect_flat).any():
            aspect_flat = replace_nans(aspect_flat, strategy="mean")
            
        mean_e = elev_flat.mean(); std_e = elev_flat.std(); skew_e = skew(elev_flat)
        elev_stats = np.vstack([
            np.full_like(elev_flat, mean_e), np.full_like(elev_flat, std_e), np.full_like(elev_flat, skew_e)
        ]).T
        
        X_meta = np.column_stack(preds + [elev_stats, slope_flat, aspect_flat])
        
        # Predicción robusta
        P = xgb_predict_full(xgb, X_meta).reshape(len(lat), len(lon))
        T = true_store[("FUSION_low", date)]
        
        # Calcular MAPE evitando NaNs
        mask_valid = ~(np.isnan(T) | np.isnan(P))
        M = np.full_like(T, np.nan)  # Inicializar con NaN
        M[mask_valid] = np.abs((T[mask_valid] - P[mask_valid])/(T[mask_valid] + 1e-5))*100
        
        # Reemplazar NaNs en mapa MAPE para visualización
        if np.isnan(M).any():
            print_progress(f"Reemplazando NaNs en mapa MAPE para visualización", level=2)
            M = np.nan_to_num(M, nan=0.0)

        # Prepare grids for plotting before use
        grid_lon, grid_lat = np.meshgrid(lon, lat)

        fig, axs = plt.subplots(1,2, figsize=(12,5), subplot_kw={'projection':ccrs.PlateCarree()})
        axs[0].set_title(f"Predicción XGB H={h}")
        pcm = axs[0].pcolormesh(grid_lon, grid_lat, P, transform=ccrs.PlateCarree(), cmap='Blues')
        boyaca_gdf.boundary.plot(ax=axs[0], edgecolor='black', transform=ccrs.PlateCarree())
        fig.colorbar(pcm, ax=axs[0], orientation='vertical', label='mm')
        axs[1].set_title(f"MAPE% XGB H={h}")
        pcm2 = axs[1].pcolormesh(grid_lon, grid_lat, M, transform=ccrs.PlateCarree(), cmap='Reds', vmin=0, vmax=np.nanpercentile(M,99))
        boyaca_gdf.boundary.plot(ax=axs[1], edgecolor='black', transform=ccrs.PlateCarree())
        fig.colorbar(pcm2, ax=axs[1], orientation='vertical', label='%')
        plt.tight_layout(); plt.show()
    except Exception as e:
        print_progress(f"❌ Error generando mapa para H={h}: {str(e)}", level=1)
        logger.exception(f"Error en visualización de mapa para H={h}")
        
 # 13.3) Mapas con modelo retrained (con manejo robusto de NaNs)
for h in range(1, OUTPUT_HORIZON+1):
    date = val_dates[h-1]
    mdl_path = MODEL_DIR/f"xgb_all_H{h}_{ref}_9features.json"
    if not mdl_path.exists():
        continue
    
    try:
        xgb = XGBRegressor(); xgb.load_model(str(mdl_path))
        
        # Reconstruir X_meta con manejo robusto
        preds = [preds_store[(f"FUSION_{b}", date)].ravel() for b in ['low','medium','high']]
        # Manejar NaNs en predicciones
        for i, b in enumerate(['low','medium','high']):
            if np.isnan(preds[i]).any():
                preds[i] = replace_nans(preds[i], strategy="mean")
                
        elev_flat = ds_full['elevation'].values.ravel()
        slope_flat = ds_full['slope'].values.ravel()
        aspect_flat = ds_full['aspect'].values.ravel()
        
        # Manejar NaNs en características topográficas
        if np.isnan(elev_flat).any():
            elev_flat = replace_nans(elev_flat, strategy="mean")
        if np.isnan(slope_flat).any():
            slope_flat = replace_nans(slope_flat, strategy="mean")
        if np.isnan(aspect_flat).any():
            aspect_flat = replace_nans(aspect_flat, strategy="mean")
            
        mean_e = elev_flat.mean(); std_e = elev_flat.std(); skew_e = skew(elev_flat)
        elev_stats = np.vstack([
            np.full_like(elev_flat, mean_e), np.full_like(elev_flat, std_e), np.full_like(elev_flat, skew_e)
        ]).T
        
        X_meta = np.column_stack(preds + [elev_stats, slope_flat, aspect_flat])
        
        # Predicción robusta
        P = xgb_predict_full(xgb, X_meta).reshape(len(lat), len(lon))
        T = true_store[("FUSION_low", date)]
        
        # Calcular MAPE evitando NaNs
        mask_valid = ~(np.isnan(T) | np.isnan(P))
        M = np.full_like(T, np.nan)  # Inicializar con NaN
        M[mask_valid] = np.abs((T[mask_valid] - P[mask_valid])/(T[mask_valid] + 1e-5))*100
        
        # Reemplazar NaNs en mapa MAPE para visualización
        if np.isnan(M).any():
            print_progress(f"Reemplazando NaNs en mapa MAPE para visualización", level=2)
            M = np.nan_to_num(M, nan=0.0)

        # Prepare grids for plotting before use
        grid_lon, grid_lat = np.meshgrid(lon, lat)

        fig, axs = plt.subplots(1,2, figsize=(12,5), subplot_kw={'projection':ccrs.PlateCarree()})
        axs[0].set_title(f"Predicción XGB H={h}")
        pcm = axs[0].pcolormesh(grid_lon, grid_lat, P, transform=ccrs.PlateCarree(), cmap='Blues')
        boyaca_gdf.boundary.plot(ax=axs[0], edgecolor='black', transform=ccrs.PlateCarree())
        fig.colorbar(pcm, ax=axs[0], orientation='vertical', label='mm')
        axs[1].set_title(f"MAPE% XGB H={h}")
        pcm2 = axs[1].pcolormesh(grid_lon, grid_lat, M, transform=ccrs.PlateCarree(), cmap='Reds', vmin=0, vmax=np.nanpercentile(M,99))
        boyaca_gdf.boundary.plot(ax=axs[1], edgecolor='black', transform=ccrs.PlateCarree())
        fig.colorbar(pcm2, ax=axs[1], orientation='vertical', label='%')
        plt.tight_layout(); plt.show()
    except Exception as e:
        print_progress(f"❌ Error generando mapa para H={h}: {str(e)}", level=1)
        logger.exception(f"Error en visualización de mapa para H={h}")


In [ ]:
# 13) Meta-modelo neuronal completo con métricas, mapas y tablas
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from scipy.stats import skew
import pandas as pd
import cartopy.crs as ccrs
from sklearn.metrics import r2_score

# Definición del modelo
class DeepMetaModel(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.model(x)

# Parámetros de entrenamiento
device   = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_sz = 64
lr       = 1e-3
epochs   = 50

# Funciones de métrica
def rmse(a, b): return np.sqrt(np.mean((a - b)**2))
def mae(a, b):  return np.mean(np.abs(a - b))
def mape(a, b): return np.mean(np.abs((a - b) / (b + 1e-5))) * 100
def r2(a, b):
    # a: predicciones, b: valores reales
    ss_res = np.sum((b - a)**2)
    ss_tot = np.sum((b - np.mean(b))**2)
    return 1 - ss_res/ss_tot if ss_tot != 0 else np.nan

def evaluate(a, b):
    return {
        'RMSE': rmse(a, b),
        'MAE': mae(a, b),
        'MAPE': mape(a, b),
        'R2': r2(a, b)
    }

# Preparar grillas para mapas
grid_lon, grid_lat = np.meshgrid(lon, lat)

# Máscaras según elevación
elev_flat = ds_full['elevation'].values.ravel()
mask_low    = elev_flat < 200
mask_mid    = (elev_flat >= 200) & (elev_flat <= 1000)
mask_high   = elev_flat > 1000

# Colecciones para métricas
global_metrics = []
elev_metrics = []
pct_metrics_list = []
trained_models = {}  # Almacenar modelos entrenados

# Bucle por cada horizonte
for h in range(1, OUTPUT_HORIZON+1):
    date = val_dates[h-1]
    print_progress(f"Processing horizon {h}, date {date}", level=1)

    # Obtener predicciones de stacking y verificar NaNs
    preds = [preds_store[(f"FUSION_{b}", date)].ravel() for b in ['low','medium','high']]
    
    # Verificar y manejar NaNs en predicciones de cada rama
    for i, branch in enumerate(['low', 'medium', 'high']):
        pred_summary = check_nans(preds[i], f"Predicción FUSION_{branch}")
        if pred_summary["has_nans"]:
            print_progress(f"⚠️ {pred_summary['nan_count']} NaNs en predicciones de {branch} ({pred_summary['nan_percentage']:.2f}%)", level=2)
            preds[i] = replace_nans(preds[i], strategy="interpolate")

    # Estadísticos globales de elevación con manejo de NaNs
    elev_flat = ds_full['elevation'].values.ravel()
    
    # Verificar NaNs en elevación
    elev_summary = check_nans(elev_flat, "Elevación")
    if elev_summary["has_nans"]:
        print_progress(f"Reemplazando {elev_summary['nan_count']} NaNs en elevación", level=2)
        elev_flat = replace_nans(elev_flat, strategy="mean")
    
    mean_e = elev_flat.mean()
    std_e = elev_flat.std()
    skew_e = skew(elev_flat)
    elev_stats = np.vstack([
        np.full_like(elev_flat, mean_e),
        np.full_like(elev_flat, std_e),
        np.full_like(elev_flat, skew_e)
    ]).T

    # Verificar NaNs en slope y aspect
    slope_flat = ds_full['slope'].values.ravel()
    aspect_flat = ds_full['aspect'].values.ravel()
    
    for arr, name in zip([slope_flat, aspect_flat], ['Slope', 'Aspect']):
        arr_summary = check_nans(arr, name)
        if arr_summary["has_nans"]:
            print_progress(f"Reemplazando {arr_summary['nan_count']} NaNs en {name}", level=2)
            if name == 'Slope':
                slope_flat = replace_nans(arr, strategy="mean")
            else:
                aspect_flat = replace_nans(arr, strategy="mean")
    
    # Construir X_meta y y_true
    X_meta = np.column_stack(preds + [elev_stats, slope_flat, aspect_flat])
    y_true = true_store[("FUSION_low", date)].ravel()
    
    # Verificar NaNs en y_true
    y_true_summary = check_nans(y_true, "Objetivo")
    if y_true_summary["has_nans"]:
        print_progress(f"Reemplazando {y_true_summary['nan_count']} NaNs en objetivo", level=2)
        y_true = replace_nans(y_true, strategy="mean")
    
    # Verificar NaNs en X_meta final
    X_meta_summary = check_nans(X_meta, "X_meta final")
    if X_meta_summary["has_nans"]:
        print_progress(f"⚠️ Aún hay {X_meta_summary['nan_count']} NaNs en X_meta, reemplazando", level=2)
        X_meta = np.nan_to_num(X_meta, nan=0.0)
    
    # Preparar DataLoader asegurando que no hay NaNs
    tx = torch.from_numpy(X_meta).float()
    ty = torch.from_numpy(y_true).float().unsqueeze(1)
    loader = DataLoader(TensorDataset(tx, ty), batch_size=batch_sz, shuffle=True)

    # Define model checkpoint path
    model_path = MODEL_DIR/f"deepmeta_H{h}_{ref}.pt"
    
    # Check if model already exists
    if model_path.exists():
        print_progress(f"Cargando modelo existente de {model_path}", level=1)
        # Load the model
        model_nn = DeepMetaModel(X_meta.shape[1]).to(device)
        model_nn.load_state_dict(torch.load(str(model_path)))
    else:
        print_progress(f"Entrenando nuevo meta-modelo neuronal para horizonte {h}", level=1)
        # Instanciar y entrenar modelo
        model_nn = DeepMetaModel(X_meta.shape[1]).to(device)
        opt = torch.optim.Adam(model_nn.parameters(), lr=lr)
        loss_fn = nn.MSELoss()
        
        # Training history
        history = {'train_loss': [], 'val_loss': []}
        best_val_loss = float('inf')
        best_state_dict = None
        
        # Split data for validation
        val_size = int(0.1 * len(X_meta))
        train_size = len(X_meta) - val_size
        train_dataset, val_dataset = torch.utils.data.random_split(
            TensorDataset(tx, ty), [train_size, val_size]
        )
        train_loader = DataLoader(train_dataset, batch_size=batch_sz, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_sz)
        
        # Training loop with validation
        for epoch in range(1, epochs+1):
            # Training phase
            model_nn.train()
            train_loss = 0
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                opt.zero_grad()
                output = model_nn(xb)
                loss = loss_fn(output, yb)
                loss.backward()
                opt.step()
                train_loss += loss.item() * len(xb)
            train_loss /= len(train_loader.dataset)
            history['train_loss'].append(train_loss)
            
            # Validation phase
            model_nn.eval()
            val_loss = 0
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb, yb = xb.to(device), yb.to(device)
                    output = model_nn(xb)
                    loss = loss_fn(output, yb)
                    val_loss += loss.item() * len(xb)
            val_loss /= len(val_loader.dataset)
            history['val_loss'].append(val_loss)
            
            # Save best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_state_dict = model_nn.state_dict().copy()
            
            if epoch % 10 == 0:
                print_progress(f"Epoch {epoch}/{epochs} — Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}", level=2)
        
        # Load best model
        model_nn.load_state_dict(best_state_dict)
        
        # Save the model
        print_progress(f"Guardando modelo en {model_path}", level=1)
        torch.save(model_nn.state_dict(), str(model_path))
        
        # Plot training history
        plt.figure(figsize=(10, 5))
        plt.plot(history['train_loss'], label='Train Loss')
        plt.plot(history['val_loss'], label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'Training History for H={h}')
        plt.legend()
        plt.savefig(IMAGE_DIR/f"deepmeta_training_h{h}.png", dpi=150)
        plt.show()
    
    # Store the model
    trained_models[h] = model_nn
    
    # Evaluación robusta con manejo de NaNs
    model_nn.eval()
    with torch.no_grad():
        preds_all = model_nn(torch.from_numpy(X_meta).float().to(device)).cpu().numpy().flatten()

    # Verificar NaNs en predicciones
    preds_summary = check_nans(preds_all, "Predicciones neuronales")
    if preds_summary["has_nans"]:
        print_progress(f"⚠️ {preds_summary['nan_count']} NaNs en predicciones, reemplazando", level=2)
        preds_all = np.nan_to_num(preds_all, nan=0.0)

    # 1) Métricas globales con manejo robusto de NaNs
    global_m = evaluate(preds_all, y_true)
    global_metrics.append({
        'horizon': h,
        'date': date,
        'RMSE': global_m['RMSE'],
        'MAE': global_m['MAE'],
        'MAPE': global_m['MAPE'],
        'R2': global_m['R2'],
        'valid_data_pct': 100 - (np.isnan(y_true).sum() / len(y_true) * 100)
    })

    # Asegurar que las máscaras de elevación no incluyan NaNs
    mask_low = (elev_flat < 200) & ~np.isnan(y_true) & ~np.isnan(preds_all)
    mask_mid = (elev_flat >= 200) & (elev_flat <= 1000) & ~np.isnan(y_true) & ~np.isnan(preds_all)
    mask_high = (elev_flat > 1000) & ~np.isnan(y_true) & ~np.isnan(preds_all)

    # 2) Métricas por elevación con manejo robusto
    if np.sum(mask_low) >= 10:
        low_m = evaluate(preds_all[mask_low], y_true[mask_low])
    else:
        low_m = {'RMSE': np.nan, 'MAPE': np.nan, 'R2': np.nan}
        
    if np.sum(mask_mid) >= 10:
        mid_m = evaluate(preds_all[mask_mid], y_true[mask_mid])
    else:
        mid_m = {'RMSE': np.nan, 'MAPE': np.nan, 'R2': np.nan}
        
    if np.sum(mask_high) >= 10:
        high_m = evaluate(preds_all[mask_high], y_true[mask_high])
    else:
        high_m = {'RMSE': np.nan, 'MAPE': np.nan, 'R2': np.nan}
    
    elev_metrics.append({
        'horizon': h, 'date': date,
        '<200m_RMSE': low_m['RMSE'], '<200m_MAPE': low_m['MAPE'], '<200m_R2': low_m['R2'],
        '200-1000m_RMSE': mid_m['RMSE'], '200-1000m_MAPE': mid_m['MAPE'], '200-1000m_R2': mid_m['R2'],
        '>1000m_RMSE': high_m['RMSE'], '>1000m_MAPE': high_m['MAPE'], '>1000m_R2': high_m['R2'],
        '<200m_valid_count': np.sum(mask_low),
        '200-1000m_valid_count': np.sum(mask_mid),
        '>1000m_valid_count': np.sum(mask_high)
    })

    # 3) Métricas por percentiles
    edges = [0,25,50,75,100]
    pcts  = np.percentile(y_true, edges)
    for i in range(4):
        lo, hi = pcts[i], pcts[i+1]
        mask_p = (y_true>=lo)&(y_true<hi)
        pm = evaluate(preds_all[mask_p], y_true[mask_p])
        pct_metrics_list.append({
            'horizon': h, 'date': date,
            f'{edges[i]}-{edges[i+1]}%_RMSE': pm['RMSE'],
            f'{edges[i]}-{edges[i+1]}%_MAPE': pm['MAPE'],
            f'{edges[i]}-{edges[i+1]}%_R2':   pm['R2']
        })

    # Gráfica True vs Predicted
    plt.figure(figsize=(5,5))
    plt.scatter(y_true, preds_all, alpha=0.3, s=2)
    mny, mxy = y_true.min(), y_true.max()
    plt.plot([mny,mxy],[mny,mxy],'k--')
    plt.title(f"True vs Pred — H={h}")
    plt.xlabel("True"); plt.ylabel("Predicted")
    plt.tight_layout();
    plt.savefig(IMAGE_DIR/f"deepmeta_scatter_h{h}.png", dpi=150)
    plt.show()

    # Mapas de Predicción y MAPE (robustos ante NaNs)
    shape = (len(lat), len(lon))
    pred_map = preds_all.reshape(shape)
    true_map = y_true.reshape(shape)
    
    # Calcular MAPE evitando divisiones por cero y NaNs
    mape_map = np.zeros_like(true_map)
    valid_mask = (true_map != 0) & ~np.isnan(true_map) & ~np.isnan(pred_map)
    mape_map[valid_mask] = np.abs((true_map[valid_mask] - pred_map[valid_mask])/(true_map[valid_mask] + 1e-5))*100
    mape_map = np.clip(mape_map, 0, 200)  # Limitar valores extremos para visualización
    
    fig,axs=plt.subplots(1,2,figsize=(12,5),subplot_kw={'projection':ccrs.PlateCarree()})
    axs[0].set_title(f"Prediction H={h}")
    pcm1=axs[0].pcolormesh(grid_lon,grid_lat,pred_map,transform=ccrs.PlateCarree(),cmap='Blues')
    boyaca_gdf.boundary.plot(ax=axs[0],edgecolor='black',transform=ccrs.PlateCarree())
    fig.colorbar(pcm1,ax=axs[0],orientation='vertical',label='mm')
    axs[1].set_title(f"MAPE% H={h}")
    pcm2=axs[1].pcolormesh(grid_lon,grid_lat,mape_map,transform=ccrs.PlateCarree(),cmap='Reds',vmin=0,vmax=np.nanpercentile(mape_map,99))
    boyaca_gdf.boundary.plot(ax=axs[1],edgecolor='black',transform=ccrs.PlateCarree())
    fig.colorbar(pcm2,ax=axs[1],orientation='vertical',label='%')
    plt.tight_layout(); 
    plt.savefig(IMAGE_DIR/f"deepmeta_maps_h{h}.png", dpi=150)
    plt.show()

# Save summary of trained models
with open(MODEL_DIR/"deepmeta_models_info.txt", "w") as f:
    f.write(f"DeepMeta Models trained on {ref}\n")
    f.write("="*50 + "\n")
    for h in trained_models:
        f.write(f"Horizon {h}: {MODEL_DIR}/deepmeta_H{h}_{ref}.pt\n")
        f.write(f"Input features: {X_meta.shape[1]}\n\n")



# --- Después del bucle: construimos los DataFrames y los mostramos ---

# 14) Tabla de métricas globales
df_global = pd.DataFrame(global_metrics)
df_global.to_csv(MODEL_DIR/f"deepmeta_global_metrics_ref{ref}.csv", index=False)
tools.display_dataframe_to_user(
    name=f"MetaNN_Global_metrics_ref{ref}",
    dataframe=df_global
)

# 15) Tabla de métricas por elevación
df_elev = pd.DataFrame(elev_metrics)
df_elev.to_csv(MODEL_DIR/f"deepmeta_elevation_metrics_ref{ref}.csv", index=False)
tools.display_dataframe_to_user(
    name=f"MetaNN_Elevation_metrics_ref{ref}",
    dataframe=df_elev
)

# 16) Tabla de métricas por percentiles (agrupada para una fila por horizonte y fecha)
df_pct = pd.DataFrame(pct_metrics_list)
df_pct_grouped = df_pct.groupby(['horizon','date'], as_index=False).max()
df_pct_grouped.to_csv(MODEL_DIR/f"deepmeta_percentile_metrics_ref{ref}.csv", index=False)
tools.display_dataframe_to_user(
    name=f"MetaNN_Percentile_metrics_ref{ref}",
    dataframe=df_pct_grouped
)

logger.info("🏁 Neural meta-model complete: metrics, tables and models saved.")

# Enhanced Training Visualization

This cell adds improved visualization of training metrics for all model types:
- Learning curves with best epoch identification
- Learning rate evolution tracking
- Convergence analysis

In [ ]:
# Enhanced visualization for model training metrics
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

# 1. Enhanced Base Model Training Visualization
print_progress("Generating enhanced training visualizations for base models", is_start=True)

def plot_enhanced_training_curves(name, history):
    """
    Generate enhanced training visualization with:
    - Loss curves (train/validation)
    - Learning rate progression
    - Best epoch marker
    
    Args:
        name: Model name
        history: Training history dictionary
    """
    if not history:
        print_progress(f"No training history available for {name}", level=1)
        return
    
    # Create figure with GridSpec for flexible layout
    fig = plt.figure(figsize=(12, 6))
    gs = gridspec.GridSpec(1, 2, width_ratios=[2, 1])
    
    # Loss curve plot
    ax1 = plt.subplot(gs[0])
    train_loss = history["loss"]
    val_loss = history["val_loss"]
    epochs = range(1, len(train_loss)+1)
    
    ax1.plot(epochs, train_loss, 'b-', label='Training loss')
    ax1.plot(epochs, val_loss, 'r-', label='Validation loss')
    
    # Find best epoch (lowest validation loss)
    best_epoch = np.argmin(val_loss) + 1
    best_loss = val_loss[best_epoch-1]
    
    # Mark best epoch
    ax1.axvline(x=best_epoch, color='green', linestyle='--', alpha=0.7)
    ax1.plot(best_epoch, best_loss, 'go', markersize=8)
    ax1.annotate(f'Best epoch: {best_epoch}\nLoss: {best_loss:.4f}', 
                 xy=(best_epoch, best_loss),
                 xytext=(best_epoch + len(epochs)*0.1, best_loss),
                 arrowprops=dict(facecolor='green', shrink=0.05, width=1.5, headwidth=8),
                 fontsize=9,
                 bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="green", alpha=0.8))
    
    ax1.set_title(f'Training Progress: {name}')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
    
    # Learning rate plot if available
    ax2 = plt.subplot(gs[1])
    if "lr" in history:
        lrs = history["lr"]
        ax2.semilogy(epochs, lrs, 'g-')
        ax2.set_title('Learning Rate')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Learning Rate (log scale)')
        ax2.grid(True, alpha=0.3)
        ax2.xaxis.set_major_locator(MaxNLocator(integer=True))
    else:
        # Plot validation vs. training loss ratio
        ratio = np.array(val_loss) / (np.array(train_loss) + 1e-10)
        ax2.plot(epochs, ratio, 'm-')
        ax2.axhline(y=1.0, color='r', linestyle='--', alpha=0.5)
        ax2.set_title('Validation/Train Loss Ratio')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Ratio')
        ax2.grid(True, alpha=0.3)
        ax2.xaxis.set_major_locator(MaxNLocator(integer=True))
    
    plt.tight_layout()
    plt.savefig(IMAGE_DIR/f"enhanced_training_{name}.png", dpi=150)
    plt.show()
    
    # Return training summary
    return {
        'model': name,
        'total_epochs': len(epochs),
        'best_epoch': best_epoch,
        'best_val_loss': best_loss,
        'final_train_loss': train_loss[-1],
        'final_val_loss': val_loss[-1],
        'early_stopping': len(epochs) < MAX_EPOCHS
    }

# Generate enhanced visualizations for all base models
training_summaries = []
for name, hist in histories.items():
    summary = plot_enhanced_training_curves(name, hist)
    if summary:
        training_summaries.append(summary)

# Create and display training summary table
if training_summaries:
    df_training = pd.DataFrame(training_summaries)
    df_training = df_training.sort_values('best_val_loss')
    df_training.to_csv(MODEL_DIR/f"training_summaries_base_models_ref{ref}.csv", index=False)
    tools.display_dataframe_to_user(
        name=f"Training_Summaries_Base_Models_ref{ref}",
        dataframe=df_training
    )

print_progress("Enhanced base model visualizations completed", is_end=True)

# 2. XGBoost Training Visualization
print_progress("Generating XGBoost training visualizations", is_start=True)

# Function to visualize XGBoost training progress
def visualize_xgb_training(model_path, horizon):
    """
    Load an XGBoost model and visualize its training progress
    
    Args:
        model_path: Path to the saved XGBoost model
        horizon: Prediction horizon
    """
    try:
        xgb = XGBRegressor()
        xgb.load_model(str(model_path))
        
        # Get feature importance
        plt.figure(figsize=(10, 6))
        feature_names = ['pred_low', 'pred_medium', 'pred_high', 
                         'elev_mean', 'elev_std', 'elev_skew', 
                         'slope', 'aspect', 'elevation']
        
        # Plot feature importance
        importances = xgb.feature_importances_
        indices = np.argsort(importances)[::-1]
        plt.bar(range(len(importances)), importances[indices], align='center')
        plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45)
        plt.title(f'XGBoost Feature Importance - Horizon {horizon}')
        plt.tight_layout()
        plt.savefig(IMAGE_DIR/f"xgb_importance_h{horizon}.png", dpi=150)
        plt.show()
        
        # Add additional visualizations if XGBoost model has training history
        if hasattr(xgb, 'evals_result') and xgb.evals_result():
            results = xgb.evals_result()
            epochs = len(results['validation_0']['rmse'])
            x_axis = range(0, epochs)
            
            # Plot training progression
            plt.figure(figsize=(12, 6))
            plt.subplot(1, 2, 1)
            plt.plot(x_axis, results['validation_0']['rmse'], 'b-', label='Training RMSE')
            if 'validation_1' in results:
                plt.plot(x_axis, results['validation_1']['rmse'], 'r-', label='Validation RMSE')
            plt.title(f'XGBoost RMSE - Horizon {horizon}')
            plt.xlabel('Boosting Rounds')
            plt.ylabel('RMSE')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            if hasattr(xgb, 'best_iteration'):
                plt.axvline(x=xgb.best_iteration, color='green', linestyle='--')
                plt.annotate(f'Best: {xgb.best_iteration}', 
                             xy=(xgb.best_iteration, min(results['validation_0']['rmse'])),
                             xytext=(xgb.best_iteration+5, min(results['validation_0']['rmse'])+0.01))
            
            # Plot feature importance on second subplot
            plt.subplot(1, 2, 2)
            plt.barh(range(len(importances)), importances[indices], align='center')
            plt.yticks(range(len(importances)), [feature_names[i] for i in indices])
            plt.title('Feature Importance')
            
            plt.tight_layout()
            plt.savefig(IMAGE_DIR/f"xgb_training_h{horizon}.png", dpi=150)
            plt.show()
        
        return {
            'horizon': horizon,
            'num_features': len(importances),
            'top_feature': feature_names[indices[0]],
            'top_importance': importances[indices[0]],
            'model_path': str(model_path)
        }
    except Exception as e:
        print_progress(f"Error visualizing XGBoost model for H={horizon}: {str(e)}", level=1)
        return None

# Visualize all XGBoost models
xgb_summaries = []
for h in range(1, OUTPUT_HORIZON+1):
    model_path = MODEL_DIR/f"xgb_all_H{h}_{ref}_9features.json"
    if model_path.exists():
        summary = visualize_xgb_training(model_path, h)
        if summary:
            xgb_summaries.append(summary)

# Create summary table
if xgb_summaries:
    df_xgb = pd.DataFrame(xgb_summaries)
    df_xgb.to_csv(MODEL_DIR/f"xgb_model_summaries_ref{ref}.csv", index=False)
    tools.display_dataframe_to_user(
        name=f"XGBoost_Model_Summaries_ref{ref}",
        dataframe=df_xgb
    )

print_progress("XGBoost training visualizations completed", is_end=True)

# 3. Neural Meta-Model Training Visualization Enhancement
print_progress("Enhancing neural meta-model training visualizations", is_start=True)

# Function to create better visualizations for neural meta-models
def visualize_neural_meta_training(model_path, history, horizon):
    """
    Create enhanced visualizations for neural meta-model training
    
    Args:
        model_path: Path to the saved model
        history: Training history dictionary 
        horizon: Prediction horizon
    """
    if not history or not model_path.exists():
        return None
    
    plt.figure(figsize=(12, 5))
    
    # Plot training curves with best epoch highlighted
    plt.subplot(1, 2, 1)
    train_loss = history['train_loss']
    val_loss = history['val_loss']
    epochs = range(1, len(train_loss)+1)
    
    plt.plot(epochs, train_loss, 'b-', label='Training Loss')
    plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
    
    best_epoch = np.argmin(val_loss) + 1
    best_loss = val_loss[best_epoch-1]
    
    plt.axvline(x=best_epoch, color='green', linestyle='--', alpha=0.7)
    plt.plot(best_epoch, best_loss, 'go', markersize=8)
    plt.annotate(f'Best: {best_epoch}\nLoss: {best_loss:.4f}', 
                 xy=(best_epoch, best_loss),
                 xytext=(best_epoch + len(epochs)*0.1, best_loss),
                 arrowprops=dict(facecolor='green', shrink=0.05, width=1.5, headwidth=8),
                 fontsize=9)
    
    plt.title(f'Neural Meta-Model Training - H={horizon}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot learning rate if available
    plt.subplot(1, 2, 2)
    if 'lr' in history:
        plt.semilogy(epochs, history['lr'], 'g-')
        plt.title('Learning Rate')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate (log scale)')
    else:
        # Calculate and plot training efficiency
        efficiency = np.array(val_loss) / (np.array(train_loss) + 1e-10)
        plt.plot(epochs, efficiency, 'm-')
        plt.axhline(y=1.0, color='r', linestyle='--', alpha=0.5)
        plt.title('Training Efficiency')
        plt.xlabel('Epoch')
        plt.ylabel('Val/Train Loss Ratio')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(IMAGE_DIR/f"neural_meta_training_h{horizon}.png", dpi=150)
    plt.show()
    
    return {
        'horizon': horizon,
        'total_epochs': len(epochs),
        'best_epoch': best_epoch,
        'best_val_loss': best_loss,
        'final_train_loss': train_loss[-1],
        'training_efficiency': val_loss[-1] / (train_loss[-1] + 1e-10),
        'model_path': str(model_path)
    }

# Placeholder for recording neural meta-model training histories
# For demonstration - in a real implementation, you'd need to capture
# these during the actual training phase
neural_meta_histories = {}
for h in range(1, OUTPUT_HORIZON+1):
    model_path = MODEL_DIR/f"deepmeta_H{h}_{ref}.pt"
    
    # Since we don't have access to the actual histories from the training code,
    # we'll just create a placeholder visualization for existing models
    if model_path.exists():
        print_progress(f"Neural meta-model exists for H={h}", level=1)
        print_progress(f"To view enhanced training curves, add history tracking during training", level=2)
        
        # In a complete implementation, you would use:
        # visualize_neural_meta_training(model_path, neural_meta_histories.get(h, {}), h)
        
        # For now, just display a message about where to find existing visualizations
        print_progress(f"Current visualizations available at: {IMAGE_DIR}/deepmeta_training_h{h}.png", level=2)

print_progress("Neural meta-model visualization enhancement completed", is_end=True)

# 4. Combined Performance Comparison
print_progress("Generating combined performance comparison", is_start=True)

# Combined visualization will be implemented here in the future
# This would compare training curves and convergence across model types

print_progress("Training visualization and analysis complete", is_end=True)